In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.tabular.all import *

In [ ]:
path = '/kaggle/input/tabular-playground-series-jul-2021/'
train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv', low_memory= False)
train.head()

In [ ]:
df_nn_cm = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv', low_memory= False)
df_nn_cm.drop(['target_benzene','target_nitrogen_oxides'], axis=1, inplace= True)
df_nn_cm.head()


In [ ]:
df_nn_bz = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv', low_memory= False)
df_nn_bz.drop(['target_carbon_monoxide','target_nitrogen_oxides'], axis=1, inplace= True)
df_nn_bz.head()

In [ ]:
df_nn_no = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv', low_memory= False)
df_nn_no.drop(['target_carbon_monoxide','target_benzene'], axis=1, inplace= True)
df_nn_no.head()

In [ ]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv', low_memory= False)
test.head()

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv',low_memory=False)
sample_submission

In [ ]:
dep_var_tcm = 'target_carbon_monoxide'
dep_var_tbz = 'target_benzene'
dep_var_tno ='target_nitrogen_oxides'

In [ ]:
df_nn_cm[dep_var_tcm]=np.log(df_nn_cm[dep_var_tcm])
df_nn_cm = add_datepart(df_nn_cm, 'date_time')

In [ ]:
df_nn_bz[dep_var_tbz]=np.log(df_nn_bz[dep_var_tbz])
df_nn_bz = add_datepart(df_nn_bz, 'date_time')

In [ ]:
df_nn_no[dep_var_tno]=np.log(df_nn_no[dep_var_tno])
df_nn_no = add_datepart(df_nn_no, 'date_time')

Extracting important columns data from my previous notebook: https://www.kaggle.com/rajatpaliwal02/tps-july-fastai-decision-tress-random-forset/edit

In [ ]:
xs_imp_cm = ['sensor_2', 'sensor_1', 'sensor_5', 'sensor_3', 'sensor_4',
       'absolute_humidity','relative_humidity']
xs_imp_bz=['sensor_2', 'sensor_3', 'sensor_4', 'absolute_humidity', 'sensor_5',
       'sensor_1']
xs_imp_no=['sensor_2', 'sensor_5', 'sensor_1', 'sensor_3', 'absolute_humidity' ,'sensor_4','deg_C', 'relative_humidity']

In [ ]:
df_nn_cm_final = df_nn_cm[list(xs_imp_cm)+[dep_var_tcm]]
df_nn_cm_final.head()

In [ ]:
df_nn_bz_final = df_nn_bz[list(xs_imp_bz)+[dep_var_tbz]]

df_nn_no_final = df_nn_no[list(xs_imp_no)+[dep_var_tno]]
df_nn_no_final.head()

In [ ]:
cont_cm,cat_cm = cont_cat_split(df_nn_cm_final, max_card=900, dep_var=dep_var_tcm)
cont_bz,cat_bz = cont_cat_split(df_nn_bz_final, max_card=900, dep_var=dep_var_tbz)
cont_no,cat_no = cont_cat_split(df_nn_no_final, max_card=900, dep_var=dep_var_tno)

In [ ]:
print(cont_bz,cat_bz),print(cont_no,cat_no), print(cont_cm, cat_cm)

In [ ]:
procs_nn = [Categorify, FillMissing, Normalize]


In [ ]:
#Look at the time period of test set, make sure it's more recent
test['date_time'].min(), test['date_time'].max()

Calculate where we should cut the validation set. We pick the most recent 'n' records in training set where n is the number of entries in test set.

In [ ]:
splits = (list(range(4902)), list(range(4902,7111)))

In [ ]:
to_nn_cm = TabularPandas(df_nn_cm_final, procs_nn, cat_cm, cont_cm,
                      splits=splits, y_names=dep_var_tcm)

In [ ]:
to_nn_bz = TabularPandas(df_nn_bz_final, procs_nn, cat_bz, cont_bz,
                      splits=splits, y_names=dep_var_tbz)

In [ ]:
to_nn_no = TabularPandas(df_nn_no_final, procs_nn, cat_no, cont_no,
                      splits=splits, y_names=dep_var_tno)

In [ ]:
dls_cm = to_nn_cm.dataloaders(1024)


In [ ]:
dls_bz= to_nn_bz.dataloaders(1024)
dls_no = to_nn_no.dataloaders(1024)

In [ ]:
y_cm = to_nn_cm.train.y
y_cm.min(),y_cm.max()

In [ ]:
y_bz = to_nn_bz.train.y
y_bz.min(),y_bz.max()

In [ ]:
y_no = to_nn_no.train.y
y_no.min(),y_no.max()

In [ ]:
learn_cm = tabular_learner(dls_cm, y_range=(-2.3,2.2), layers=[500,250],
                        n_out=1, metrics=rmse,loss_func=nn.MSELoss())

In [ ]:
learn_bz = tabular_learner(dls_bz, layers=[500,250],y_range=(-2.3,3.73),
                        n_out=1, metrics=rmse,loss_func=nn.MSELoss())
learn_no = tabular_learner(dls_no, layers=[1000,800],y_range=(0.64,6.79),
                        n_out=1, metrics=rmse,loss_func=nn.MSELoss())

In [ ]:
learn_cm.model

In [ ]:
learn_bz.model

In [ ]:
learn_no.model

In [ ]:
learn_cm.lr_find()

In [ ]:
learn_bz.lr_find()

In [ ]:
learn_no.lr_find()

In [ ]:
learn_cm.fit_one_cycle(5, 1e-2)


In [ ]:
learn_cm.fit_one_cycle(8, 1e-3)

In [ ]:
learn_cm.fit_one_cycle(8, 1e-4)

In [ ]:
learn_cm.fit_one_cycle(5, 1e-5)

In [ ]:
learn_cm.fit_one_cycle(5, 1e-6)

In [ ]:
learn_cm.fine_tune(5)

In [ ]:
learn_bz.fit_one_cycle(5, 1e-2)

In [ ]:
learn_bz.fit_one_cycle(5, 1e-3)

In [ ]:
learn_bz.fit_one_cycle(5, 1e-4)

In [ ]:
learn_bz.fit_one_cycle(5, 1e-5)

In [ ]:
learn_bz.fine_tune(5)

In [ ]:
learn_no.fit_one_cycle(5, 1e-2)

In [ ]:
learn_no.fit_one_cycle(5, 1e-3)

In [ ]:
learn_no.fit_one_cycle(6, 1e-4)

In [ ]:
learn_no.fit_one_cycle(7, 1e-5)

In [ ]:
learn_no.fine_tune(6)

In [ ]:
dl_cm = learn_cm.dls.test_dl(test[cont_cm+cat_cm])
pred_cm = learn_cm.get_preds(dl=dl_cm)[0].numpy()[:,0]
dl_bz = learn_bz.dls.test_dl(test[cont_bz+cat_bz])
pred_bz = learn_bz.get_preds(dl=dl_bz)[0].numpy()[:,0]
dl_no = learn_no.dls.test_dl(test[cont_no+cat_no])
pred_no = learn_no.get_preds(dl=dl_no)[0].numpy()[:,0]


In [ ]:
sample_submission[sample_submission.columns[1]] = np.exp(pred_cm)
sample_submission[sample_submission.columns[2]] = np.exp(pred_bz)
sample_submission[sample_submission.columns[3]] = np.exp(pred_no)
sample_submission

In [ ]:
sample_submission.to_csv('submission_nn9.csv', index=False)